```{contents}
```
## **Review Gate in LangGraph**

A **Review Gate** in LangGraph is a **control node that pauses execution to allow validation, correction, or approval of the current state** before the graph is allowed to continue.
It introduces **human or automated governance** into otherwise autonomous workflows.

In production systems, Review Gates are the **primary mechanism for safety, quality control, and compliance**.

---

### **1. Motivation**

LLM systems are powerful but **non-deterministic**.
Certain operations **must not proceed** without verification:

| Scenario                  | Risk                |
| ------------------------- | ------------------- |
| Financial decisions       | Monetary loss       |
| Legal text generation     | Liability           |
| Code deployment           | System failure      |
| Medical reasoning         | Patient safety      |
| Autonomous tool execution | Irreversible damage |

Review Gates solve this by **forcing explicit approval**.

---

### **2. Conceptual Model**

```
… → LLM Reasoning → Review Gate → (Approve | Revise | Reject) → Continue / Stop
```

The gate **halts execution** until a decision is made.

---

### **3. How Review Gates Work in LangGraph**

LangGraph has no special "ReviewGate" object.
Instead, it is implemented using:

* **Interrupt nodes**
* **Conditional routing**
* **State inspection**
* **Human-in-the-loop callbacks**

---

### **4. State Design**

```python
class State(TypedDict):
    draft: str
    approved: bool
    feedback: str
```

---

### **5. Review Gate Implementation**

### **Review Node**

```python
def review_node(state: State):
    print("\n--- REVIEW REQUIRED ---")
    print(state["draft"])
    
    decision = input("Approve? (y/n): ")
    
    if decision.lower() == "y":
        return {"approved": True}
    else:
        feedback = input("Enter feedback: ")
        return {"approved": False, "feedback": feedback}
```

---

### **Routing Logic**

```python
def review_router(state: State):
    if state["approved"]:
        return "finalize"
    return "revise"
```

---

### **Graph Construction**

```python
builder.add_node("review", review_node)
builder.add_node("revise", revise_node)
builder.add_node("finalize", finalize_node)

builder.add_conditional_edges(
    "review",
    review_router,
    {"finalize": "finalize", "revise": "revise"}
)
```

---

### **Execution Flow**

```
Generate Draft
      ↓
Review Gate (human input)
   ↙        ↘
Revise     Finalize
```

---

### **6. Automated Review Gate**

Review gates can be **model-based** instead of human:

```python
def automated_review(state):
    verdict = critic_llm.invoke(f"Is this correct? {state['draft']}")
    return {"approved": "yes" in verdict.lower()}
```

Used for:

* Code linting
* Fact checking
* Policy compliance
* Safety filtering

---

### **7. Production Patterns**

| Pattern          | Description                  |
| ---------------- | ---------------------------- |
| Hard Gate        | Must approve before progress |
| Soft Gate        | Logs warnings, continues     |
| Multi-Level Gate | Multiple reviewers           |
| Time-Bound Gate  | Auto-fail after timeout      |
| Escalation Gate  | Senior approval on rejection |

---

### **8. Why Review Gates Are Critical**

Without Review Gates:

* Autonomous systems become unsafe
* Errors propagate
* No accountability

With Review Gates:

* Systems become **governable**
* Errors are **caught early**
* Compliance becomes **enforceable**

---

### **9. Typical Use Cases**

* Financial automation
* Contract drafting
* Medical decision support
* AI code deployment
* Compliance workflows
* Autonomous agent supervision

---

### **10. Mental Model**

A Review Gate transforms LangGraph from a **workflow engine** into a **controlled decision system**.

> **No irreversible action without approval.**

### Demonstration

In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph, END

# ----------------------------
# 1. State Schema
# ----------------------------
class State(TypedDict):
    draft: str
    approved: bool
    feedback: str

# ----------------------------
# 2. Draft Generator
# ----------------------------
def generate(state: State):
    return {"draft": "This is the initial AI-generated report.", "approved": False}

# ----------------------------
# 3. Review Gate (Human-in-the-loop)
# ----------------------------
def review(state: State):
    print("\n--- REVIEW GATE ---")
    print("Draft:", state["draft"])
    
    decision = input("Approve? (y/n): ").strip().lower()
    
    if decision == "y":
        return {"approved": True}
    else:
        feedback = input("Provide feedback: ")
        return {"approved": False, "feedback": feedback}

# ----------------------------
# 4. Revision Node
# ----------------------------
def revise(state: State):
    revised = f"{state['draft']} | Revision based on feedback: {state['feedback']}"
    return {"draft": revised}

# ----------------------------
# 5. Routing Logic
# ----------------------------
def route(state: State):
    if state["approved"]:
        return "finalize"
    return "revise"

# ----------------------------
# 6. Final Node
# ----------------------------
def finalize(state: State):
    print("\n--- FINAL OUTPUT ---")
    print(state["draft"])
    return {}

# ----------------------------
# 7. Build Graph
# ----------------------------
builder = StateGraph(State)

builder.add_node("generate", generate)
builder.add_node("review", review)
builder.add_node("revise", revise)
builder.add_node("finalize", finalize)

builder.set_entry_point("generate")

builder.add_edge("generate", "review")

builder.add_conditional_edges("review", route, {
    "revise": "revise",
    "finalize": "finalize"
})

builder.add_edge("revise", "review")
builder.add_edge("finalize", END)

graph = builder.compile()

# ----------------------------
# 8. Run
# ----------------------------
graph.invoke({"draft": "", "approved": False, "feedback": ""})



--- REVIEW GATE ---
Draft: This is the initial AI-generated report.


Approve? (y/n):  y



--- FINAL OUTPUT ---
This is the initial AI-generated report.


{'draft': 'This is the initial AI-generated report.',
 'approved': True,
 'feedback': ''}